<a href="https://colab.research.google.com/github/sigvehaug/CAS-Applied-Data-Science/blob/master/Module-1/M1_Web_Scraping/1-Tutorials/2-APIs_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with APIs #

---


<font color='red'>
Stars* mark more advanced (or less crucial) topics. <font color = 4e1585>SIDENOTES <font color='red'>provide background information for those who are interested in what happens "behind the scenes". 


---


In the previous part, we learned how to automatically retrieve pages from the internet through **web scraping**. We sent HTTP requests with the  ``requests`` library and used the ``BeautifulSoup`` library to parse and work with the HTML code from the response we got. For many of the more popular websites such as Wikipedia, Youtube, Twitter or many newspapers, there is a more direct way to retrieve the information we need: through so-called **APIs**. In this tutorial we will learn how to gather web data through APIs. We will use the Wikipedia API to illustrate the process.

To follow allong with the tutorial, you will have to import the ``requests`` and the ``BeautifulSoup`` library:


In [ ]:
import requests
from bs4 import BeautifulSoup

##Getting help

We are selective in this tutorial and only discuss elements that we believe are most important for the purpose of this class. If you want more details, you can consult, for example, the **Python Standard Library Reference** at https://docs.python.org/3/library/ or the **Language Reference** at https://docs.python.org/3/reference/. But be warned: the amount of detail in these sources can be overwhelming. For **quick and easy-to-understand overviews** of different topics see, for example, https://www.w3schools.com/python/. Here are some specific references for today's tutorial:

Working with APIs in Python:
* https://realpython.com/python-api/
* https://www.dataquest.io/blog/python-api-tutorial/

JSON: 
*  https://www.w3schools.com/js/js_json_intro.asp

If you get stuck or don't remember how to do something, it is usually a good idea to **Google** your problem. Python has a large (and fast-growing) community and you will probably find answers to most of your questions online (e.g. on **Stack Overflow** or in a **Youtube tutorial**).

## What is an API?

Suppose you have a (large) list of addresses and need the respective coordinates. If you wanted to do this manually, you could use Google Maps through your browser. But how could you automate this? You could try to generate the URLs for each request and fetch the coordinates from the pages. This may be feasable, but the pages you will request contain a lot of information and graphical "overhead" you are not interested in, making the process very tedious and inefficient (and web scrapers often get blocked by such services, as they are mostly comercial or have limited capacity). Fortunately, many providers offer an easier way to access the information in their databases: through APIs (e.g. tha Google Maps API).

So what is an API? API stands for **Application Programming Interface** and it is a very broad concept. You can think of it as an interface that is not meant for humans but for computer programs or applications. APIs allow different applications or programs to interact with each other. For example, the different programs installed on your computer may communicate with the operating system of your computer through APIs (e.g. when saving a file).

  
In our context of gathering data from the web, we are mostly dealing with so-called **Web APIs**. They allow you to access content in a more **structured** way than scraping HTML pages. Web APIs consist of **a set of URLs through which the data is made accessible so that other applications/programs (for example your Python script) can work with it conveniently**. They also allow to take more control over the content you will receive. When providers offer an API, it just means that they’ve built a set of URLs that return structured data — meaning the responses won’t contain the presentational overhead that is required for a graphical user interface like a website. Web APIs have become so important that many people just say API when they are referring to a Web API.

This probably still sounds very abstract. We'll walk you through some examples with the Wikipedia API to give you a more concrete idea of how APIs work. That being said, we still download and process data via an URL, only the URL and data structure will change a bit.

## Making API requests with the ``requests`` library

How can we gather data through an API? In practice, **retrieving data through an API is quite similar to retrieving data through web scraping**. You will also have to **figure out the correct URL** for your request. In fact, you can type this URL into the browser — **instead of a designed web page, you will directly see structured data**. For example, this URL takes you to the structured data from the cat article on Wikipedia: https://en.wikipedia.org/w/api.php?action=parse&page=Cat&format=json

If we want to request the cat article through the API, we can thus just type:

In [ ]:
r = requests.get("https://en.wikipedia.org/w/api.php?action=parse&page=Cat&format=json")
r.text

Let's take a closer look at the URL we used: The first part of the URL (https://en.wikipedia.org/w/api.php) is an **endpoint** of the API. You can think of endpoints as the base URLs of the API. After the ``?`` we have the **query string** (as we know already from the web scraping tutorial). It specifies the action we want to perform. Usually, it is more convenient to enter the parameters for the query string as an argument. The **``params`` parameter of the ``get`` function** allows you to do this (the order of the parameters typically does not matter):


In [ ]:
ENDPOINT = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "parse",
    "page": "Cat",
    "format": "json",
}

r = requests.get(url=ENDPOINT, params=PARAMS)
r.text

To see that this makes exactly the same request, you can take a look the URL we just called:

In [ ]:
r.url

So far, so good. But how did we figure out that https://en.wikipedia.org/w/api.php?action=parse&page=Cat&format=json was the URL we needed to request? Many APIs have an extensive **documentation** that explain how to make different types of requests. These documentations can sometimes be overwhelming and it is usually a good idea to try to find some examples for the type of request you want to make (e.g. on Stack Overflow or within the documentation). Let's have a look at the documentation for the Wikipedia API: 

* https://www.mediawiki.org/wiki/API:Main_page

We can see that there are different endpoints. For example, there is one endpoint for each language. The endpoint for English Wikipedia is: 
* https://en.wikipedia.org/w/api.php 

Now we will have to figure out what parameters to choose (i.e. how to construct the query string). If you scroll down on the page, you will see that there are many options as to how the parameters could be set. If you click through the documentation (or Google something like "parse page Wikipedia API") you may eventually get to this page that provides some useful examples as to how the content of a page could be retrieved: 
* https://www.mediawiki.org/wiki/API:Parsing_wikitext

There, you will find example code similar to the one we used above.



><font color = 4e1585> SIDENOTE: APIs often allow you to do many different things. You can retrieve information, but sometimes you can also add, change or delete information. Many Web APIS are so-called RESTful APIs, meaning that they implement the standard HTTP methods: GET, POST, PUT and DELETE. For data science projects, you are typically only interested in GET requests (as you don't want to change anything on the respective websites). </font>

## Working with JSON data

We mentioned that APIs allow you to access *structured data*. What exactly does this mean? Typically, the data you will retrieve through an API will be formatted as **JSON**. JSON stands for JavaScript Object Notation; it is a data exchange format based on human-readable text (as opposed to binary data formats) that is commonly used for data transportation over the web. Let's take a look at some JSON code:



```json
{
  "name": "Peter",
  "age": 56,
  "married": false,
  "children": [
    {
      "name": "Nina",
      "age": 19,
      "educational degree": "High school"
    },
    {
      "name": "Mary",
      "age": 14,
      "educational degree": null
    }
  ]
}
```



><font color = 4e1585> SIDENOTE: If you want to know if something is valid (correctly formatted) JSON you can search for 'JSON lint' or 'JSON linter' and use one of the online tools, e.g. https://jsonlint.com. A lint or linter is a tool to check for errors in code (see https://en.wikipedia.org/wiki/Lint_(software) for more details)</font>

You may have noticed that JSON does not look so different from a Python dictionary. The great thing about JSON is that it can easily be **converted into a (nested) Python dictionary**. Let's parse the response we got from our Wikipida API request:

In [ ]:
data = r.json()

In [ ]:
print(data)
type(data)

We have now created a (large) Python dictionary. How can we **access the different parts** of it? If you **type the URL of your request into your browser**, you can expore the nested structure of the JSON code (and thus of your Python dictionary):


In [ ]:
r.url # this will display the URL; copy the URL to your browser and see what happens


Similarly, it is usually a good idea to explore the **keys** of the dictionary:

In [ ]:
data.keys()

We only have one key at the top level (``parse``). Let's explore its value to see what we have on the next level:  

In [ ]:
data["parse"].keys()

Now we can access different types of information about the page. Let's access the title:

In [ ]:
data["parse"]["title"]

Let's now try to access the sections:

In [ ]:
sections = data["parse"]["sections"]
sections

We got a list of dictionaries containing information on each section. If we are only interested in the heading of each section, we could write a list comprehension to extract it. The "line" key in each dictionary appears to contain the section heading.

In [ ]:
[section["line"] for section in sections]

In this way, you can access different parts of your dictionary and extract the information on the page (e.g. internal and external links, images etc.). The dictionary (within "parse") **also contains a "text" key where the HTML code of the article is stored** (within the "*" key). You can convert it to a BeautifulSoup object and search it based on HTML tags (see notebook on web scraping): 

In [ ]:
# Get html text 
html_text = BeautifulSoup(data["parse"]["text"]["*"])
type(html_text)

In [ ]:
# get opening paragraph
html_text.find_all("p")[1]

---

>  <font color='teal'> **In-class exercise**:
Request the *Dog* page on Wikipedia (formatted as JSON) through the API and convert the response to a Python dictionary. 

>  <font color='teal'> Print the title and all links to other Wikipedia articles.



---



## Making advanced requests with the Wikipedia API

The Wikipedia API allows you to do much more than just retrieving a single page. For example, you can search for **pages that match with certain words** (full text search in title or content: https://www.mediawiki.org/wiki/API:Search). Let's retrieve all pages about cats:

In [ ]:
# Perform page search 
URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "format": "json",
    "list": "search",
    "srsearch": "cat",
    "srlimit": "max"
}

r = requests.get(url=URL, params=PARAMS)

# Convert to dictionary
DATA = r.json()

In [ ]:
DATA.keys()

In [ ]:
# Navigate through dictionary
print(DATA.keys())
print(DATA["query"].keys())

In [ ]:
# Get titles of all cat pages that were found
pages = DATA["query"]["search"]
cat_pages = [page["title"] for page in pages]
print(cat_pages) 
len(cat_pages)

Another interesting thing you can do is to **find pages based on coordinate locations** (see: https://www.mediawiki.org/wiki/API:Geosearch). Let's find all Wikipedia pages of places that are close (<1km) to the main building of University of Bern:

In [ ]:
# Perform page search 
URL = "https://de.wikipedia.org/w/api.php"  # Change to German Wikipedia!

PARAMS = {
    "action": "query",
    "format": "json",
    "list": "geosearch",
    "gscoord": "46.950519|7.438109",
    "gslimit": "max",
    "gsradius": 1000 
}

r = requests.get(url=URL, params=PARAMS)

# Convert to dictionary
DATA = r.json()

In [ ]:
PLACES = DATA['query']['geosearch']
[place["title"] for place in PLACES]

There are many more things you can do with the Wikipedia API. Some of them would be extremely difficult (or impossible) to achieve through web scraping (e.g. the two examples in this section). Check out the documentation (or Google your idea) to find out!

## Using the ``wikipedia`` module for simple requests

For many of the more popular APIs, someone has written a **Python Module** that allows you to work with the API more conveniently. For example, there is a **``wikipedia`` module** to work with the Wikipedia API (see here for a documentation: https://pypi.org/project/wikipedia/):

In [ ]:
# install and import wikipedia module
!pip install wikipedia # When ! is being added at the beginning the statement is performed in the command-line!
import wikipedia

In [ ]:
# Search for pages about cats
wikipedia.search("Cat") # only first 10 results by default

In [ ]:
# Get contents of cat page
wikipedia.page('Cat', auto_suggest=False).content

Such modules can be very convenient if you want to perform simple requests. However, they are often not very flexible and may not always allow you to make the requests you would like to perform. <font color='red'>Moreover, the quality may vary (anyone can program and upload such a module) as the following example shows:</font>

In [ ]:
wikipedia.page('Cat').content # Search for 'Cat' shows result for 'Hat', as auto_suggest parameter is True by default.

---

>  <font color='teal'> **In-class exercise**:
Retrieve 20 Wikipedia pages that are located within a 2 km radius from the main train station in Bern (46.949722°, 7.439444°)



---



## Working with APIs in practice

Now we have seen many different ways to retrieve data from the web and you might be asking yourself: When should I do web scraping? When should I use an API? When should I use a Python module (that accesses an API)?

Or course, this depends on your project and on your preferences, but here's a rule of thumb: 

* Whenever there is an **API**, it is usually easier to work with it than to scrape the pages. Also, some data is only really accessible through APIs (e.g. if you want to retrieve all tweets about a certain topic on Twitter). So if there's an API, it's usually (but of course not always) a good idea to use it!

* If there's a **Python library** for the API, it is a good idea to check it out and see what it can do. If it suits your needs, it may save you a lot of work (reading the API documentation etc.). However, some people prefer to work directly with the API to keep full control over their implementation.

* **Web scraping** is useful when there is no API, which is the case for most websites. For some tasks, you may also find it easier to just scrape the pages instead of trying to find out how the API works.



When you start working with other APIs you will notice that each one works a bit differently. For example, many APIs require **authentication** (e.g. Twitter API, Google Maps API, Facebook API etc.). This means that you will first have to create a "developer account" and will then get some credentials you have to include into your request. You will have to read the documentation of the API (or some blog or answer on Stack Overflow) to find out how this works for the API you are interested in. Also, not all APIs give you **free access** to all the data. For example, the Google Maps API will start to charge you a price after a certain amount of requests, so be careful with your implementation.

APIs are becoming increasingly common and larger providers usually have one (or several). Here are some **APIs you may find useful for a research project**:

| Name | Link | Python Module(s) |
| --- | --- | --- |
| Wikipedia API | https://www.mediawiki.org/wiki/API:Main_page | wikipedia, Wikipedia-API |
| Twitter API | https://developer.twitter.com/en/docs/twitter-api | twitter, tweepy |
 Google Maps API| https://developers.google.com/maps/documentation | python-gmaps|
| LinkedIn API|https://www.linkedin.com/developers/|python-linkedin-v2|
| New York Times API|https://developer.nytimes.com/apis| pynytimes|
| The Guardian API |https://open-platform.theguardian.com/documentation/|| 
| COVID-19 API |https://covid19api.com/ |
| Google Trends (no official API) | https://towardsdatascience.com/google-trends-api-for-python-a84bc25db88f | pytrends|

